In [ ]:
!git clone https://github.com/mrcuongtroll/PENLI.git

Cloning into 'PENLI'...
remote: Enumerating objects: 394, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 394 (delta 86), reused 80 (delta 34), pack-reused 230
Receiving objects: 100% (394/394), 102.22 KiB | 2.62 MiB/s, done.
Resolving deltas: 100% (207/207), done.


In [ ]:
!git clone https://github.com/OanaMariaCamburu/e-SNLI.git

Cloning into 'e-SNLI'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 220 (delta 4), reused 0 (delta 0), pack-reused 211
Receiving objects: 100% (220/220), 70.43 MiB | 18.32 MiB/s, done.
Resolving deltas: 100% (100/100), done.
Updating files: 100% (96/96), done.


In [ ]:
import os
os.chdir('./PENLI')

In [ ]:
!git pull
!ls
!pip install -r requirements.txt
!pip install sentencepiece

Already up to date.
configs		download.py	model		  shell_scripts  train.py
data		get_dataset.py	README.md	  test.py	 train_rl.py
definitions.py	logger		requirements.txt  trainer	 utils
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=491e97cbecea5dc8eda11934371ccddc2b8be7ca75cdd258aade461cc7a87165
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
!python definitions.py

In [ ]:
!mkdir /content/PENLI/datasets/e-SNLI
!cp /content/e-SNLI/dataset/esnli_train_1.csv /content/PENLI/datasets/e-SNLI
!cp /content/e-SNLI/dataset/esnli_train_2.csv /content/PENLI/datasets/e-SNLI
!cp /content/e-SNLI/dataset/esnli_dev.csv /content/PENLI/datasets/e-SNLI
!cp /content/e-SNLI/dataset/esnli_test.csv /content/PENLI/datasets/e-SNLI

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from utils.config import ConfigParser
from utils.evaluate import evaluate_model
import model.model as model_module
import data.data_loader as data_loader
import logger.logger as logger_module
import torch.nn as nn
import torch
import logging
from tqdm import tqdm_notebook

logger = logging.getLogger()

In [ ]:
test_dataset_path = './datasets/e-SNLI/esnli_test.csv'
config_path = './configs/default_ed.json'
checkpoint_path = '/content/drive/MyDrive/Thesis/default_t5_base/rl/checkpoint_best.pt'
state_dict = 'state_dict'
device = 'cuda'

In [ ]:
config = ConfigParser(config_path)
config.init_obj(logger_module, "logging")
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = config.init_obj(model_module, "model")
logger.info(f"-----> Loading model checkpoint from {checkpoint_path}...")
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint[state_dict])
model.to(device)
logger.info(f"-----> Done.")

[2023-07-07 11:07:34,547][root][INFO] - -----> Loading model checkpoint from /content/drive/MyDrive/Thesis/default_ed/rl/checkpoint_best.pt...
[2023-07-07 11:08:37,893][root][INFO] - -----> Done.


In [ ]:
config['test_loader']['kwargs']['file_path'] = test_dataset_path
logger.info(f"Testing the model on {config['test_loader']['kwargs']['file_path']}...")
test_loader = config.init_obj(data_loader, 'test_loader', tokenizer=model.tokenizer, use_explanation=True)

[2023-07-07 11:08:37,904][root][INFO] - Testing the model on ./datasets/e-SNLI/esnli_test.csv...


In [ ]:
def get_prob_stats(model, dataloader, target, truth):
    tokenizer = model.tokenizer
    lut = {
        'true': tokenizer.encode('true')[0],
        'false': tokenizer.encode('false')[0],
        'neutral': tokenizer.encode('neutral')[0]
    }
    label_ids = [tokenizer.encode('true')[0],
                 tokenizer.encode('false')[0],
                 tokenizer.encode('neutral')[0]]
    probs = {
        'true': 0,
        'false': 0,
        'neutral': 0,
        'others': 0
    }
    count = 0
    with torch.no_grad():
        for batch_idx, batch in tqdm_notebook(enumerate(dataloader),
                                              total=len(dataloader),
                                              desc='Processing...'):
            assert dataloader.dataset.model_type == 2, "Set dataset's model_type to 2 when using T5."
            input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
            input_ids, attention_mask, labels = (input_ids.to(device),
                                                 attention_mask.to(device),
                                                 labels.to(device)
                                                 )
            outputs, full_outputs = model(input_ids=input_ids,
                                          attention_mask=attention_mask,
                                          labels=labels)
            loss = full_outputs.loss
            outputs = torch.exp(outputs)
            pred = torch.argmax(outputs, dim=-1)
            for i in range(len(labels)):
                label = [labels[i][j].item() for j in range(len(labels[i])) if labels[i][j].item() != -100]
                out = [pred[i][j].item() for j in range(len(labels[i])) if labels[i][j].item() != -100]
                if label[0] == lut[truth] and out[0] == lut[target]:
                    count += 1
                    probs['true'] += outputs[i, 0, lut['true']].item()
                    probs['false'] += outputs[i, 0, lut['false']].item()
                    probs['neutral'] += outputs[i, 0, lut['neutral']].item()
                    others_prob = outputs[i, 0, :].sum() - outputs[i, 0, lut['true']].item() - outputs[i, 0, lut['false']].item() - outputs[i, 0, lut['neutral']].item()
                    probs['others'] += others_prob.item()
    for label_type in probs:
        probs[label_type] /= count
    return probs

In [ ]:
probs = get_prob_stats(model, test_loader, target='neutral', truth='neutral')

<ipython-input-12-e6106549fa6c>:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_idx, batch in tqdm_notebook(enumerate(dataloader),


Processing...:   0%|          | 0/614 [00:00<?, ?it/s]

In [ ]:
print(probs)

{'true': 0.0011490377575369495, 'false': 0.002098237558046355, 'neutral': 0.9966449083524085, 'others': 0.00010793372555711775}


In [ ]:
# criterion = config.init_obj(nn, "loss")
# results = evaluate_model(model=model,
#                          data_loader=test_loader,
#                          criterion=criterion,
#                          config=config,
#                          use_explanation=False,
#                          device=device)
# logger.info(f"*********************FINISHED EVALUATING MODEL**********************")
# logger.info(results)

[2023-07-07 11:29:52,735][utils.evaluate][INFO] - Evaluating [960/9824 (10%)] | Loss: 0.57739 
[2023-07-07 11:29:58,887][utils.evaluate][INFO] - Evaluating [1936/9824 (20%)] | Loss: 0.64343 
[2023-07-07 11:30:04,868][utils.evaluate][INFO] - Evaluating [2912/9824 (30%)] | Loss: 0.66231 
[2023-07-07 11:30:10,996][utils.evaluate][INFO] - Evaluating [3888/9824 (40%)] | Loss: 0.64755 
[2023-07-07 11:30:17,083][utils.evaluate][INFO] - Evaluating [4864/9824 (50%)] | Loss: 0.65950 
[2023-07-07 11:30:23,206][utils.evaluate][INFO] - Evaluating [5840/9824 (59%)] | Loss: 0.66120 
[2023-07-07 11:30:29,410][utils.evaluate][INFO] - Evaluating [6816/9824 (69%)] | Loss: 0.65777 
[2023-07-07 11:30:35,405][utils.evaluate][INFO] - Evaluating [7792/9824 (79%)] | Loss: 0.66836 
[2023-07-07 11:30:41,640][utils.evaluate][INFO] - Evaluating [8768/9824 (89%)] | Loss: 0.66290 
[2023-07-07 11:30:47,650][utils.evaluate][INFO] - Evaluating [9744/9824 (99%)] | Loss: 0.66008 
[2023-07-07 11:30:48,107][root][INFO] - *